# Chapter 18: Graphs

In [1]:
from collections import deque, defaultdict, namedtuple
from copy import deepcopy

# Test Graphs
test_graph_1 = {'A': set(['F', 'B', 'C']),
              'B': set(['A', 'D', 'E', 'G']),
              'C': set(['A', 'F']),
              'D': set(['B', "G"]),
              'E': set(['B', 'F']),
              'F': set(['C', 'E']),
              'G': set()}

test_graph_2 = deepcopy(test_graph_1)
test_graph_2["Z"] = "Y"

## Breadth First Search

### Vanilla BFS

In [67]:
def bfs(graph, root, directed=False):
    """
    Performs breadth first search on the given graph.
    """
    processed, discovered = set(), set(root)
    parent = defaultdict(None)
    q = deque(root)
    
    while q:
        u = q.popleft()
        process_vertex_early(u)
        for v in graph[u]:
            if v not in processed or directed:
                process_edge(u, v)
            if v not in discovered:
                q.append(v)
                discovered.add(v)
                parent[v] = u
        processed.add(u)
        process_vertex_late(u)
    print("Parents: {}".format(parent))
        
def process_vertex_early(v):
    """
    Performs computation on a vertex just before exploring its adjacency list
    """
    print("Processed vertex: {}".format(v))
    
    
def process_edge(u, v):
    """
    Performs computation on an edge the first time it is discovered
    """
    print("Processed edge: {}{}".format(u, v))
    
    
def process_vertex_late(v):
    """
    Performs computation on a vertex after exploring its adjacency list
    """
    pass


# Tests
print(test_graph_1)
bfs(test_graph_1, "A")

{'A': {'B', 'C', 'F'}, 'B': {'E', 'D', 'G', 'A'}, 'C': {'A', 'F'}, 'D': {'B', 'G'}, 'E': {'B', 'F'}, 'F': {'E', 'C'}, 'G': set()}
Processed vertex: A
Processed edge: AB
Processed edge: AC
Processed edge: AF
Processed vertex: B
Processed edge: BE
Processed edge: BD
Processed edge: BG
Processed vertex: C
Processed edge: CF
Processed vertex: F
Processed edge: FE
Processed vertex: E
Processed vertex: D
Processed edge: DG
Processed vertex: G
Parents: defaultdict(None, {'B': 'A', 'C': 'A', 'F': 'A', 'E': 'B', 'D': 'B', 'G': 'B'})


### Applications of BFS

Connected Components

In [51]:
def connected_components(graph):
    """
    Returns the number of connected components in the given graph
    """
    discovered = set()
    count = 0
    
    def bfs(root):
        discovered.add(root)
        q = deque(root)
        while q:
            u = q.popleft()
            for v in graph.get(u, set()):
                if v not in discovered:
                    discovered.add(v)
                    q.append(v)
    
    for vertex in graph:
        if vertex not in discovered:
            count += 1
            bfs(vertex)
    
    return count
            
# Tests
assert connected_components(test_graph_1) == 1
assert connected_components(test_graph_2) == 2

Two-coloring graphs

In [2]:
def bipartite(graph, color1, color2):
    """
    Return True iff the given graph is bipartite
    """
    discovered, processed = set(), set()
    color = defaultdict(None)
    bipartite = True
    
    def complement(color):
        if color == color1: return color2
        elif color == color2: return color1

    def process_edge(u, v):
        if color[u] == color[v]:
            nonlocal bipartite
            bipartite = False
        color[v] = complement(color(u))
        
    def bfs(root):
        discovered.add(root)
        q = deque(root)
        while q:
            u = q.popleft()
            processed.add(u)
            for v in graph.get(u, set()):
                if v not in processed:
                    process_edge(u, v)
                if v not in discovered:
                    discovered.add(v)
                    q.append(v)
    
    for vertex in graph:
        if vertex not in discovered:
            color[vertex] = color1
            bfs(vertex)
    
    return bipartite

# Tests
bipartite(test_graph, "WHITE", "BLACK")

## Depth First Search

### Vanilla DFS

In [ ]:
def dfs(graph, root, directed = False):
    """
    Performs depth first search on the given graph.
    """
    discovered, processed = set(), set()
    
    def recur(u):
        discovered.add(u)
        process_vertex_early(u)
        
        for v in graph[u]:
            if v not in discovered:  # Tree edge
                process_edge(u, v)
                recur(v)
            elif v not in processed or directed:  # Back edge
                process_edge(u, v)

        process_vertex_late(u)
        processed.add(u)
    
    recur(root)
        
                  
def process_vertex_early(v):
    """
    Performs computation on a vertex just before exploring its adjacency list
    """
    print("Processed vertex: {}".format(v))

    
def process_edge(u, v):
    """
    Performs computation on an edge the first time it is discovered
    """
    print("Processed edge: {}{}".format(u, v))
    

def process_vertex_late(v):
    """
    Performs computation on a vertex after exploring its adjacency list
    """
    pass

# Tests
print(test_graph_1)
dfs(test_graph_1, "A")

## 18.1 Search a maze

In [11]:
WHITE, BLACK = range(2)

Coordinate = namedtuple("Coordinate", ("x", "y"))

def search_maze(maze, start, end):
    """
    Returns a possible path from `start` to `end`
    """
    path = []
    
    def invalid_vertex(v):
        """
        Returns True if the given vertex is out of the matrix or if it is BLACK
        """
        return not((0 <= v.x < len(maze)) and 
                  (0 <= v.y < len(maze)) and
                  maze[v.x][v.y] == WHITE) 
    
    def get_neighbors(v):
        """
        Returns an iterator of this vertex's neighbors
        """
        for n in (Coordinate(v.x+1, v.y), Coordinate(v.x-1, v.y), Coordinate(v.x, v.y+1), Coordinate(v.x, v.y-1)):
            yield n
        
    def dfs(u):
        """
        Perform DFS
        """
        if invalid_vertex(u):
            return False
        
        # Indicate that vertex is discovered
        maze[u.x][u.y] = BLACK
        
        # Process vertex early
        path.append(u)
        if(u == end):
            return True
        
        # Process neighbours
        for v in get_neighbors(v):
            if bfs(v): return True
        
        # Process vertex late
        path.pop()
        return False
    
    if dfs(start): return path
    
    return []

## 18.2 Paint a Boolean Matrix

In [6]:
Coordinate = namedtuple("Coordinate", ("x", "y"))

def get_neighbors(v):
    """
    Returns an iterator of this vertex's neighbors
    """
    for n in (Coordinate(v.x+1, v.y), Coordinate(v.x-1, v.y), Coordinate(v.x, v.y+1), Coordinate(v.x, v.y-1)):
        yield n
            
            
def flip_color_dfs(M, root):
    """
    Flips the color of the region associated with M[x, y] by performing DFS.
    """
    color = M[x][y]
    M[x][y] = 1 - M[x][y]  # Indicate that vertex is discovered
    
    # Process neighbors
    for v in get_neighbors(root):
        if ((0 <= v.x < len(M)) and (0 <= v.y < len(M[v.x])) and (M[v.x][v.y] == color)):
            flip_color_dfs(M, v)
    
    return M
    

def flip_color_bfs(M, root):
    """
    Flips the color of the region associated with M[x, y] by performing BFS.
    """
    q = deque(root)
    
    color = M[root.x][root.y]
    M[root.x][root.y] = 1 - M[root.x][root.y]  # Indicate that vertex is discovered
    
    while q:
        u = q.popleft()
        for v in get_neighbors(u):  # Process neighbors
            if ((0 <= v.x < len(M)) and (0 <= v.y < len(M[v.x])) and (M[v.x][v.y] == color)):
                M[v.x][v.y] = 1 - M[v.x][v.y]  # Indicate that vertex is discovered
                q.append(v)
    
    return M    

## 18.3 Compute Enclosed Regions

In [ ]:
BLACK, WHITE = range(2)
Coordinate = namedtuple("Coordinate", ("x", "y"))

def fill_enclosed_regions(M):
    """
    Fills the enclosed regions of the given matrix BLACK.
    """
    
    def valid_vertex(v):
        """
        Returns True if the given vertex is out of the matrix or if it is BLACK
        """
        return not((0 <= v.x < len(M)) and 
                   (0 <= v.y < len(M)) and
                    M[v.x][v.y] == WHITE) 
    
    def get_neighbors(v):
        """
        Returns an iterator of this vertex's neighbors
        """
        for n in (Coordinate(v.x+1, v.y), Coordinate(v.x-1, v.y), Coordinate(v.x, v.y+1), Coordinate(v.x, v.y-1)):
            yield n
            
    def boundary_vertex(v):
        """
        Returns True if the given point is on the boundary of matrix M
        """
        return ((v.x == 0 or v.x == len(M) - 1) or
                (v.y == 0 or v.y == len(M[0]) - 1))
    
    def dfs(u):
        """
        Performs DFS and returns True if a boundary point is found from `u`.
        """
        if boundary_vertex(u):
            return True
        
        M[u.x][u.y] = BLACK  # Indicate that vertex is discovered
        
        # Process neighbors
        reach_boundary = False
        for v in get_neighbors(u):
            if valid_vertex(v):
                reach_boundary |= dfs(v)
        
        # Process vertex late
        if reach_boundary: M[u.x][u.y] = WHITE
        
        return reach_boundary
    
    # Perform DFS over all WHITE squares
    for i in range(len(M)):
        for j in range(len(M[i])):
            if M[i][j] == WHITE:
                dfs(Coordinate(i, j))
    
    return M
        
    
# Tests
A = [[0, 0, 0, 0],
     [1, 0, 1, 0],
     [0, 1, 1, 0],
     [0, 0, 0, 0]]

A_res = [[0, 0, 0, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 0],
         [0, 0, 0, 0]]
assert fill_enclosed_regions(A) == A_res

## 18.7 Transform one string to another

Look at the book for another approach

In [30]:
def transform_string(D, s, t):
    """
    Returns the minimum number of steps for `s` to transform to `t` if possible.
    Returns None otherwise
    """
    Vertex = namedtuple("Vertex", ("string", "distance"))
    
    def differ_by_one(word1, word2):
        """
        Returns True if the edit distance of the given words is 1.
        """
        return len(set(word1) - set(word2)) == 1
        
    def build_graph():
        """
        Builds a graph of 1-edit distance from dictionary `D`.
        """
        g = defaultdict(set)
        for word in D:
            for other_word in D:
                if word != other_word and differ_by_one(word, other_word):
                    g[word].add(other_word)
        return g
    
    def bfs(graph):
        """
        Perform BFS.
        """
        root = Vertex(s, 0)
        q, discovered = deque(), set(s)
        q.append(root)
        while q:
            u = q.popleft()
            if u.string == t:  # Process vertex early
                return u.distance
            for v in graph[u.string]:
                if v not in discovered:
                    discovered.add(v)  # Indicate that vertex is discovered
                    q.append(Vertex(v, u.distance + 1))
        
        return None
            
    graph = build_graph()
    return bfs(graph)

# Tests
D = {"bat", "cot", "dog", "dag", "dot", "cat"}
assert transform_string(D, "cat", "dog") == 3